Spark RDD mapReduce

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/02/24 02:14:00 WARN Utils: Your hostname, codespaces-c6e2dd resolves to a loopback address: 127.0.0.1; using 10.0.3.141 instead (on interface eth0)
25/02/24 02:14:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/02/24 02:14:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

#Express the following SQL statement using RDD


    SELECT 
        date_trunc('hour', lpep_pickup_datetime) AS hour, 
        PULocationID AS zone,
    
        SUM(total_amount) AS amount,
        COUNT(1) AS number_records
    FROM
        green
    WHERE
        lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY
        1, 2
        

In [5]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

#Will build out functions needed before executing

rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) \
    .show()


In [6]:
from datetime import datetime

In [7]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [8]:
rows = rdd.take(10)
row = rows[0]

In [9]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

In [10]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [11]:
#Reduce by key

def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [12]:
from collections import namedtuple

In [13]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [14]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [15]:
from pyspark.sql import types

In [16]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [17]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [18]:
df_result.show()

[Stage 2:============================================>              (3 + 1) / 4]

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-20 15:00:00|  67|              79.5|    3|
|2020-01-05 16:00:00|  82| 500.4700000000002|   33|
|2020-01-16 08:00:00|  41| 736.1399999999996|   54|
|2020-01-20 15:00:00|  75| 608.9999999999999|   47|
|2020-01-17 21:00:00|  74|            594.87|   39|
|2020-01-03 09:00:00|  61|            142.21|    9|
|2020-01-13 19:00:00| 223|213.83000000000004|   19|
|2020-01-16 13:00:00|  14|163.73999999999998|    6|
|2020-01-12 15:00:00|  82| 265.7900000000001|   29|
|2020-01-08 16:00:00|  72|            130.67|    7|
|2020-01-03 11:00:00|  51| 305.9599999999999|   10|
|2020-01-30 09:00:00|  22|114.80000000000001|    4|
|2020-01-31 18:00:00| 260| 439.3100000000002|   27|
|2020-01-10 20:00:00|  66|            405.88|   21|
|2020-01-31 21:00:00|  41| 588.1600000000001|   40|
|2020-01-23 22:00:00| 166|            297.84|   18|
|2020-01-30 

In [19]:
df_result.write.parquet('tmp/green-revenue')

Spark RDD mapPartition

In [20]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [21]:
import pandas as pd
pd.DataFrame.iteritems = pd.DataFrame.items

In [22]:
rows = duration_rdd.take(10)

In [23]:
df = pd.DataFrame(rows, columns=columns)

In [24]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [25]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [26]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [27]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [28]:
df_predicts.select('predicted_duration').show()

[Stage 8:>                                                          (0 + 1) / 1]

+------------------+
|predicted_duration|
+------------------+
|63.849999999999994|
|              40.0|
|              6.35|
|              6.25|
| 9.200000000000001|
|               3.8|
|16.599999999999998|
|             11.05|
|               4.5|
|              30.5|
|               8.7|
|5.8999999999999995|
|              11.0|
|              15.2|
|              4.25|
|25.299999999999997|
|7.8500000000000005|
|              34.0|
| 5.300000000000001|
|              6.15|
+------------------+
only showing top 20 rows

